<a href="https://colab.research.google.com/github/gatetub/Team-Synergy-Intel/blob/main/Gradio_Model_%2B_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Initialisation**


# **Pre-requisite:**

*   Gemini Token
*   Two csv files for respective category and their paths in content of collab

In [ ]:
!pip install gradio
!pip install -q -U google-generativeai

In [18]:
import json
import pandas as pd

In [19]:
df=pd.read_csv("/content/tech_reviews.csv")    # run this to analyse technical reviews' sentiment

In [31]:
df=pd.read_csv("/content/reviews_user.csv")  # run this to analyse user reviews' sentiment

In [ ]:
df

In [33]:

product_names = df['product_name'].unique()

data_sets = {}
for product_name in product_names:

  data_sets[product_name] = df[df['product_name'] == product_name]


In [ ]:
product_names

# Gemini Model

In [35]:
import google.generativeai as genai
import json
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GEMINI_TOKEN')  # give your gemini access key as GEMINI_TOKEN in collab secrets
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')
# model = genai.GenerativeModel('gemini-1.5-pro')

In [36]:
def call_gemini(text):
  response = model.generate_content(text)
  return response.text

In [37]:
def data(text1):
  text_column_values = data_sets[text1]['text'].tolist()
  single_string = ' '.join(text_column_values)
  review_rating_map = {}

# Iterate through each row in the DataFrame
  for index, row in df.iterrows():
  # Get the review date and rating
    review_date = row["review_date"]
    review_rating = row["review_rating"]

  # Add the mapping to the dictionary
    review_rating_map[review_date] = review_rating

  jsonl_data = [json.dumps({"review_date": k, "review_rating": v}) for k, v in review_rating_map.items()]

# Write the JSONL data to a file
  with open("review_ratings.jsonl", "w") as f:
      for line in jsonl_data:
        f.write(line + "\n")

  prompt = f"""
    You are a sentiment analysis expert tasked with analyzing customer reviews to track trends in positive and negative sentiment over the years. The goal is to gain insights about customer perception of the processor over time.
    only provide me your output do not show any code .Customer reviews are provided between three backticks below.
    ```
    {jsonl_data}
    ```
    """
  print(jsonl_data)
  gem_data = call_gemini(prompt)

  return gem_data

# **Web Interface**

1.   User Reviews
2.   Technical Reviews




**Common functions for user and technical reviews**

In [38]:
import pandas as pd


product_names_list = product_names.tolist()
def get_all_data(product_name):
    filtered_df = df[df['product_name'] == product_name]
    if not filtered_df.empty:
        texts = filtered_df['text'].tolist()
        cleaned_texts = filtered_df['cleaned_text'].tolist()
        review_ratings = filtered_df['review_rating'].tolist()
        return texts, cleaned_texts, review_ratings
    else:
        return ["Product not found."], ["Product not found."], ["Product not found."]


In [39]:
def display_table(product_name):
    texts, cleaned_texts, review_ratings = get_all_data(product_name)

    if texts == ["Product not found."]:
        table = "<p>Product not found.</p>"
    else:
        table = "<table><tr><th>Review</th><th>Sentiment Indicators</th><th>Review Type</th></tr>"
        for text, cleaned_text, rating in zip(texts, cleaned_texts, review_ratings):
            table += f"<tr><td>{text}</td><td>{cleaned_text}</td><td>{rating}</td></tr>"
        table += "</table>"

    return table

# **User Review Visualisation**

In [ ]:
import gradio as gr

iframe_html = """
<iframe title="product_power_bi_dashboard" width="100%" height="273.5"
src="https://app.powerbi.com/view?r=eyJrIjoiN2VjNzY0NjEtYTc1Ny00MTk2LWIxNzUtNzM3M2NiNjlmMzNlIiwidCI6Ijg0YzMxY2EwLWFjM2ItNGVhZS1hZDExLTUxOWQ4MDIzM2U2ZiIsImMiOjZ9"
frameborder="0" allowFullScreen="true"></iframe>
"""

iframe_html1 = """
<iframe title="v3_updated" width="100%" height="273.5" src="https://app.powerbi.com/view?r=eyJrIjoiMmQ0ZGM4YzQtOTNmNi00NDRmLWE1Y2UtOTQ5ZjVlMTMxODNjIiwidCI6Ijg0YzMxY2EwLWFjM2ItNGVhZS1hZDExLTUxOWQ4MDIzM2U2ZiIsImMiOjZ9" frameborder="0" allowFullScreen="true"></iframe>
"""

with gr.Blocks() as demo:
    gr.Markdown("# Intel Products Sentiment Analysis from Online User Reviews")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Insights on User reviews collected ")
            gr.HTML(iframe_html)
        with gr.Column(scale=1):
            gr.Markdown("### Sentiment Analysis on User Review")
            gr.HTML(iframe_html1)

    with gr.Row():
        with gr.Column(scale=1):
            product_dropdown = gr.Dropdown(choices=product_names_list, label="Select Product")
        with gr.Column(scale=1):
            product_output = gr.Textbox(label="Output using Gemini Model", interactive=False)

        product_dropdown.change(
            fn=data,
            inputs=product_dropdown,
            outputs=product_output,
        )


    with gr.Row():
        with gr.Column(scale=1):
            table_output = gr.HTML()
            product_dropdown.change(
                fn=display_table,
                inputs=product_dropdown,
                outputs=table_output,
            )

demo.launch(share=True, debug=True)

# **Technical Review Visualisation And Model Prediction**

In [28]:
def dummy_function(text1):
  text_column_values = data_sets[text1]['text'].tolist()
  single_string = ' '.join(text_column_values)
  review_rating_map = {}

# Iterate through each row in the DataFrame
  for index, row in df.iterrows():
  # Get the review date and rating
    review_date = row["review_date"]
    review_rating = row["text"]

  # Add the mapping to the dictionary
    review_rating_map[review_date] = review_rating

  jsonl_data = [json.dumps({"review_date": k, "review_rating": v}) for k, v in review_rating_map.items()]

# Write the JSONL data to a file
  with open("review_ratings.jsonl", "w") as f:
      for line in jsonl_data:
        f.write(line + "\n")

  prompt = f"""
    You are a sentiment analysis expert tasked with analyzing customer reviews to track the technical specifiaction and its performance over the years. The goal is to gain insights about customer perception of the processor over time.
    only provide me your output do not show any code .Customer reviews are provided between three backticks below.
    ```
    {jsonl_data}
    ```
    """
  print(jsonl_data)
  gem_data1 =call_gemini(prompt)

  return gem_data1

In [ ]:
import gradio as gr

# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Intel Products Sentiment Analysis from Online Technical Reviews")
    gr.Markdown("### Sentiment Analysis on Technical Reviews")

    with gr.Row():
        with gr.Column(scale=1):
            product_dropdown = gr.Dropdown(choices=product_names_list, label="Select Product")
        with gr.Column(scale=1):
            iframe_html3 = """
            <iframe title="tech_reviews_v3_final" width="600" height="273.5" src="https://app.powerbi.com/view?r=eyJrIjoiZmY5MThiMTktY2UzNi00OWI4LWI2M2QtZjczNjcwNGU3MmM4IiwidCI6Ijg0YzMxY2EwLWFjM2ItNGVhZS1hZDExLTUxOWQ4MDIzM2U2ZiIsImMiOjZ9" frameborder="0" allowFullScreen="true"></iframe>
            """
            gr.HTML(iframe_html3)


    with gr.Row():
        with gr.Column(scale=1):
            dummy_output = gr.Textbox(label="Technical Insights", interactive=False)
        with gr.Column(scale=1):
            product_output = gr.Textbox(label="Output using Gemini Model", interactive=False)

        product_dropdown.change(
            fn=dummy_function,
            inputs=product_dropdown,
            outputs=dummy_output
        )

        product_dropdown.change(
            fn=data,
            inputs=product_dropdown,
            outputs=product_output
        )

    with gr.Row():
        table_output = gr.HTML(label="Product Reviews and Cleaned Texts")

    product_dropdown.change(
        fn=display_table,
        inputs=product_dropdown,
        outputs=table_output
    )

demo.launch(share=True, debug=True)
